Neste notebook, consta o código para anotar os frames de um vídeo por meio do modelo definitivo. 

**OBS**: alguns alertas podem ser levantados durante sua execução.

# Imports

In [ ]:
import sys
import PIL
from project_utils.fo_dataset_pipeline import *
from project_utils.draw_detections import *

In [ ]:
from tqdm import trange

# Carregando o modelo

In [ ]:
# caminho para o modelo definitivo
model_path = '../faster_rcnn/final_test/model_0/mmdet.faster_rcnn_resnext101_32x4d_fpn_1x.pth'

# carrega o modelo 
model_type, model, img_size, class_map = get_model_from_checkpoint(model_path)

# define a lista de transformações na imagem para inferência pelo modelo conforme os parâmetros de treinamento
infer_tfms = get_tfms(img_size=img_size, tfms_type = 'valid')

# Carregando o Vídeo

In [ ]:
# solicita o nome de arquivo do vídeo
video_filename = input('Digite o nome do arqivo de vídeo: ')

# define o nome do novo arquivo de vídeo com os frames anotados
new_video_filename = f'new_{video_filename.split(".")[0]}.mp4'

# inicia a captura de vídeo
video = cv2.VideoCapture(video_filename)

# verifica se o vídeo abriu corretamente
if video.isOpened() == False:
    # caso não tenha aberto, encerra o programa
    print('Erro ao abrir arquivo!')
    sys.exit() 

# defina a largura e altura dos frames
frame_width = int(video.get(3))
frame_height = int(video.get(4))

# define o codec (no caso, será MP4, mas podem ser outros)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# writer para novo vídeo
new_video = cv2.VideoWriter(new_video_filename, fourcc, 30, (frame_width, frame_height))


# Anotando o Vídeo

In [ ]:
# número total de frames
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print('Iniciando a escrita dos frames ...')

for _ in trange(total_frames):
    # lê o próximo frame
    ret, frame = video.read()

    if ret == True: 
        # converte de BGR para RGB
        pil_array = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # converte array em objeto PIL
        pil_frame = PIL.Image.fromarray(pil_array)

        # inferência
        preds = model_type.end2end_detect(pil_frame, infer_tfms, model, class_map=class_map, detection_threshold=0.5)
        
        # anota a frame
        new_frame = draw_bb(frame, preds)

        # salva o frame
        new_video.write(new_frame)
        
    else: 
        # caso não haja mais frames válidos para ler, encerre o laço
        break


'''
while True: 

    if current_frame % 10 == 0: 
        # a cada 10 frames, exibe o total de frames já inferidos pelo nmodelo
        print(f'frames: {current_frame}/{total_frames}')
    
    # lê o próximo frame
    ret, frame = video.read()

    if ret == True: 
        # converte de BGR para RGB
        pil_array = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # converte array em objeto PIL
        pil_frame = PIL.Image.fromarray(pil_array)

        # inferência
        preds = model_type.end2end_detect(pil_frame, infer_tfms, model, class_map=class_map, detection_threshold=0.5)
        
        # desenha a caixa delimitadora no novo frame
        new_frame = draw_bb(frame, preds)

        # salva o frame
        new_video.write(new_frame)
        
    else: 
        # caso não haja mais frames para ler, encerre o laço
        break
        
    current_frame += 1
'''

# Salvando o Vídeo

In [ ]:
video.release()
new_video.release()